<h2>Market Research - chicken exports</h2>

<a href="https://openclassrooms.com/en/" >Openclassrooms</a>, Data Analyst Course
<br>Project 5 - Michael Orange


<hr>

A chicken producer is looking for new export markets. All countries are considered. 
The core objective is to assist the company in targeting the markets with the highest potential. 

'Grouping' the countries in 'clusters' with similar caracteristics is expected. Variables are: 
- country population, 
- population growth, 
- food diet: calory supply per capita, protein supply per capita, 
- proportion of animal protein in the mix. 

Additional relevant variable might be included (ex. GDP per capita)

<hr>

<h2>Preparation Dataset data_country</h2>

<h3>Import Datasets</h3>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

Data are collected from The Food and Agriculture Organization Corporate Statistical Database (FAOSTAT). The FAOSTAT website disseminates statistical data collected and maintained by the Food and Agriculture Organization (FAO).

FAOSTAT - Data collected (2013 and 2017 are selected for the below analysis): analysis is based on 2017 figures, with a comparaison from 2013 data. 
2017 is the most recent comprehensive data from FAOSTAT.

- Datasets Food Balance Animal and Vegetal
- Dataset Population
- Dataset Gross Domestic Product (GDP)
- Dataset Trade Mix - Chicken 

In [ ]:
#import food datasets
veg2013 = pd.read_csv("data/raw/vegetal2013.csv")
ani2013 = pd.read_csv("data/raw/animal2013.csv")
veg2017 = pd.read_csv("data/raw/vegetal2017.csv")
ani2017 = pd.read_csv("data/raw/animal2017.csv")
ani2013["origin"] = "animal"
veg2013["origin"] = "vegetal"
ani2017["origin"] = "animal"
veg2017["origin"] = "vegetal"

#import population
pop = pd.read_csv("data/raw/FAOSTAT_data_Population_2013-2017.csv")

# import GDP
gdp = pd.read_csv("data/raw/FAOSTAT_data_MacroIndicators_20132017.csv")

# import chicken trade
chicken_trade = pd.read_csv("data/raw/FAOSTAT_data_Trade_Chicken_20132017.csv")

<h3>Preparation food dataset</h3>


In [ ]:
# group food datatsets
temp = [veg2013, ani2013, ani2017, veg2017]
temp = pd.concat(temp, ignore_index=True)

# delete ani2013, veg2013, ani2017, veg2017
del ani2013, veg2013, ani2017, veg2017

temp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'element', 'item_code', 'item',
                'xx4', "year", "unit", "value", 'xx5', 'flag_description', 'origin', 'xx6']

data = temp.pivot_table(index=["year", "country_code", "country", "origin", "item_code", \
                               "item"], columns = ["element"], values=["value"], aggfunc=sum)

# rename columns
data.columns = ['domestic_supply_quantity','export_quantity','fat_supply_quantity_gcapitaday','feed',
                'food','food_supply_kcalcapitaday','food_supply_quantity_kgcapitayr','import_quantity','losses','other_uses','processing',
                'production', 'protein_supply_quantity_gcapitaday', 'residuals', 'seed','stock_variation','tourist_consumption']

data = data.reset_index()

Data for China are duplicated. 
<br>data for 'China' = sum of the splitted data for 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan'.
- deletion of 'China' data (code 351)
- 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan' data are kept in order to preserve a better granularity.



In [ ]:
# remove doublon from China (code_country 351)
data = data.loc[data.country_code != 351]

# remove Bermuda and Brunei - no information for 2017
data = data.loc[data.country != 'Bermuda']
data = data.loc[data.country != 'Brunei Darussalam']

<h3>Population data</h3>

- Population per country
- Growth of the population (%) - 2017 vs 2013

In [ ]:
# population
pop.columns = ["xx","xx2","country_code","country", 'xx3', 'xx4', 'xx5','xx6', 
               'xx7', 'year',"xx8","population_total_thousand","xx9", 'xx10', 'xx11']

pop = pop.drop(["xx","xx2",'xx3','xx4','xx5', 'xx6', 'xx7','xx8', 'xx9', 'xx10', 'xx11'], axis = 1)
pop.reset_index(inplace=True)

# merge data and pop
data = pd.merge(data, pop, how='left')

data_country = pop.pivot_table(index=['country_code', 'country'], columns = ['year'], values=['population_total_thousand'])
data_country.columns = ['pop_2013_thousand', 'pop_2017_thousand']
data_country.reset_index(inplace=True)

#growth population
data_country['pop_growth_percent'] = (data_country['pop_2017_thousand'] / data_country['pop_2013_thousand'] - 1) *100

<h3>Diet caracterisitics : proteins and calories </h3>

- Food supply kcal per capita per day - per country
- Protein supply g per capita per day - per country
- Growth of the protein supply per capita per country (%) - 2017 vs 2013

In [ ]:
temp = data.pivot_table(index=['country_code', 'country'], columns=['year'], \
                        values=['food_supply_kcalcapitaday',  'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['food_supply_kcalcapitaday_2013', 'food_supply_kcalcapitaday_2017', \
                'protein_supply_gcapitaday_2013', 'protein_supply_gcapitaday_2017']
temp = temp.reset_index()

temp['protein_supply_capita_growth_percent'] = (temp['protein_supply_gcapitaday_2017'] / temp['protein_supply_gcapitaday_2013'] - 1) * 100

data_country = pd.merge(temp, data_country, how='left')

- Food supply kcal per country (kcal)
- Protein supply per country (kg)
- Growth of the protein supply per country (%) - 2017 vs 2013

In [ ]:
NB_DAYS_YEAR = 365 

data_country['food_supply_kcal_2013'] = data_country['food_supply_kcalcapitaday_2013'] * NB_DAYS_YEAR * data_country['pop_2013_thousand'] * 1000
data_country['food_supply_kcal_2017'] = data_country['food_supply_kcalcapitaday_2017'] * NB_DAYS_YEAR * data_country['pop_2017_thousand'] * 1000

data_country['protein_supply_kg_2013'] = data_country['protein_supply_gcapitaday_2013']/1000 * NB_DAYS_YEAR *data_country['pop_2013_thousand']*1000
data_country['protein_supply_kg_2017'] = data_country['protein_supply_gcapitaday_2017']/1000 * NB_DAYS_YEAR *data_country['pop_2017_thousand']*1000

data_country['protein_supply_growth_percent'] = (data_country['protein_supply_kg_2017'] /data_country['protein_supply_kg_2013'] - 1) * 100

Animal proteins per capita

- Protein supply g per capita per day from animals - per country
- Growth of the protein supply per capita per country (%) from animals - 2017 vs 2013


In [ ]:
# protein per capita from animals
temp = data.loc[data['origin']=='animal'].pivot_table(index=['country_code', 'country'], columns = ['year'], \
                                                      values=[ 'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['protein_supply_animal_gcapitaday_2013', 'protein_supply_animal_gcapitaday_2017']
temp.reset_index(inplace=True)

temp['protein_supply_capita_animal_growth_percent'] = \
    (temp['protein_supply_animal_gcapitaday_2017'] / temp['protein_supply_animal_gcapitaday_2013'] - 1) * 100

data_country = pd.merge(data_country, temp, how='left')


Animal proteins total

- Protein supply kg from animals - per country
- Growth of the protein supply per country (%) from animals - 2017 vs 2013

In [ ]:
# protein supply from animals
data_country['protein_supply_animal_kg_2013'] = \
    data_country['protein_supply_animal_gcapitaday_2013']/1000 * NB_DAYS_YEAR *data_country['pop_2013_thousand']*1000

data_country['protein_supply_animal_kg_2017'] = \
    data_country['protein_supply_animal_gcapitaday_2017']/1000 * NB_DAYS_YEAR *data_country['pop_2017_thousand']*1000

data_country['protein_supply_animal_growth_percent'] = (data_country['protein_supply_animal_kg_2017'] /data_country['protein_supply_animal_kg_2013'] - 1) * 100

Prevalence of animal proteins in the diet mix - per country

In [ ]:
# prevalence of proteins from animals

data_country['prevalence_protein_animal_capita_percent_2013'] = \
    data_country['protein_supply_animal_gcapitaday_2013'] / data_country['protein_supply_gcapitaday_2013'] * 100
data_country['prevalence_protein_animal_capita_percent_2017'] = \
    data_country['protein_supply_animal_gcapitaday_2017'] / data_country['protein_supply_gcapitaday_2017'] * 100

data_country['prevalence_protein_animal_percent_2013'] = data_country['protein_supply_animal_kg_2013'] / data_country['protein_supply_kg_2013'] * 100
data_country['prevalence_protein_animal_percent_2017'] = data_country['protein_supply_animal_kg_2017'] / data_country['protein_supply_kg_2017'] * 100

In [ ]:
# growth of the prevalence of proteins from animals
data_country['prevalence_protein_animal_capita_growth_percent'] = \
    (data_country['prevalence_protein_animal_capita_percent_2017'] /data_country['prevalence_protein_animal_capita_percent_2013'] - 1) * 100

data_country['prevalence_protein_animal_growth_percent'] = \
    (data_country['prevalence_protein_animal_percent_2017'] /data_country['prevalence_protein_animal_percent_2013'] - 1) * 100

<h3>Gross Domestic Product (GDP) per capita</h3>

- GDP per capita (USD)
- GDP Growth - 2017 vs 2013

In [ ]:
gdp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'xx4', 'xx5', 'item', 'xx6', "year", "xx7", "value", 'xx8', 'xx9', 'xx10']
gdp = gdp.loc[gdp['item'] == 'Gross Domestic Product per capita'].pivot_table(index=["year", "country_code", "country"], columns = ['item'], values=['value'])
gdp.columns = ['gdp_percapita_usd']

gdp = gdp.pivot_table(index=["country_code", "country"], columns = ['year'], values=['gdp_percapita_usd'])
gdp.columns = ['gdp_percapita_usd_2013', 'gdp_percapita_usd_2017']
gdp.reset_index(inplace=True)

data_country = pd.merge(data_country, gdp, how='left')

GDP of Taiwan is missing. 
Informaiton collected from the International Monetary Fund (IMF).
- GDP per capita 2013 : 43 739 USD
- GDP per capita 2017 : 50 593 USD

Source: https://bit.ly/2U2251b

In [ ]:
data_country.loc[data_country['country'] == 'China, Taiwan Province of', 'gdp_percapita_usd_2013'] = 43739
data_country.loc[data_country['country'] == 'China, Taiwan Province of', 'gdp_percapita_usd_2017'] = 50593

In [ ]:
#growth of the GDP
gdp['gdp_percapita_growth'] = (gdp['gdp_percapita_usd_2017'] / gdp['gdp_percapita_usd_2013'] - 1) * 100

<h3>Poultry</h3>



Poultry are domesticated avian species that can be raised for eggs, meat and/or feathers. The term “poultry” covers a wide range of birds, from indigenous and commercial breeds of chickens to Muscovy ducks, mallard ducks, turkeys, guinea fowl, geese, quail, pigeons, ostriches and pheasants. 

- In 2017, chickens accounted for some 92 percent of the world’s poultry population, followed by ducks (5 percent), and turkeys (2 percent). 
- Chickens contribute 89 percent of world poultry meat production, followed by turkeys with 5 percent, ducks with 4 percent and geese and guinea fowl with 2 percent. The rest comes from other poultry species.
- Chickens provide 92 percent of world egg production.

source: http://www.fao.org/poultry-production-products/production/en/

There is no granular data about chicken exclusively in the dataset 'animal' but the data for the category 'Poultry' (in which chicken is accounting around 90% of the total) give us a fair indicator of the chicken market in each country. 

Poultry composition: Meat chicken, Fat liver prepared (foie gras), Meat chicken canned, Meat,duck, Meat goose and guinea fowl, Meat turkey.

In [ ]:
temp = data.loc[(data['item'] == 'Poultry Meat')|(data['item'] == 'Eggs')].pivot_table(index=['country_code', 'country'], columns=['year'], \
                                                            values=['domestic_supply_quantity', 'food', 'import_quantity', 'protein_supply_quantity_gcapitaday'], aggfunc=sum)

temp.columns = ['domestic_supply_poultry_tons_2013', 'domestic_supply_poultry_tons_2017', 'food_poultry_tons_2013',  'food_poultry_tons_2017',\
                'import_poultry_tons_2013', 'import_poultry_tons_2017', 'protein_supply_poultry_gcapitaday_2013', 'protein_supply_poultry_gcapitaday_2017']

temp.reset_index(inplace=True)
temp

#convert 1000 tons to tons
temp['domestic_supply_poultry_tons_2013'] = temp['domestic_supply_poultry_tons_2013'] * 1000
temp['domestic_supply_poultry_tons_2017'] = temp['domestic_supply_poultry_tons_2017'] * 1000
temp['food_poultry_tons_2013'] = temp['food_poultry_tons_2013'] * 1000
temp['food_poultry_tons_2017'] = temp['food_poultry_tons_2017'] * 1000
temp['import_poultry_tons_2013'] = temp['import_poultry_tons_2013'] * 1000
temp['import_poultry_tons_2017'] = temp['import_poultry_tons_2017'] * 1000

data_country = pd.merge(data_country, temp, how='left')

In [ ]:
#growth Poultry+Eggs protein supply
data_country['food_poultry_growth'] = (data_country['food_poultry_tons_2017'] / data_country['food_poultry_tons_2013'] - 1) * 100
#growth Poultry+Eggs imports
data_country['import_poultry_growth'] = (data_country['import_poultry_tons_2017'] / data_country['import_poultry_tons_2013'] - 1) * 100

<h3>Chicken Importations</h3>

Data are from the food and agricultural trade dataset collected, processed and disseminated by FAO. The data is mainly provided by UNSD, Eurostat, and other national authorities as needed.
- Import quantity of 'chicken' per country
- Import value of 'chicken' per country

The category 'chicken' is made of 'chicken', 'meat, chicken', 'meat, chicken, canned', 'offals, liver chicken'. 
Offals liver chicken was not available in the 'Poultry' section.  

In [ ]:
chicken_trade.columns =['xx', 'xx2', 'country_code', 'country', 'partner_country_code', 'partner_country', 'xx6', \
                        'element', 'xx8', 'xx9', 'xx10', 'year', 'unit', 'value', 'xx13', 'xx14', 'xx15']

# there is a disruptancy on the name of the UK between the FAO trade dataset and other FAO datasets
chicken_trade.loc[chicken_trade['country'] == 'United Kingdom', 'country'] = 'United Kingdom of Great Britain and Northern Ireland'
chicken_trade.loc[chicken_trade['partner_country'] == 'United Kingdom', 'partner_country'] = 'United Kingdom of Great Britain and Northern Ireland'

In [ ]:
#Imports of chicken per country - in quantity and value

chicken_import = chicken_trade.loc[((chicken_trade['element'] == 'Import Value') | (chicken_trade['element'] == 'Import Quantity')) \
                                  & (chicken_trade['unit'] != '1000 Head')]

chicken_import = chicken_import.pivot_table(index=["country_code", "country"], columns=['year', 'element'], values=['value'], aggfunc=sum)

chicken_import.columns = ['import_chicken_quantity_tons_2013', 'import_chicken_value_thousand_usd_2013', 'import_chicken_quantity_tons_2017', \
                         'import_chicken_value_thousand_usd_2017']
chicken_import.reset_index(inplace=True)

data_country = pd.merge(data_country, chicken_import, how='left')

In [ ]:
country_no_report_import_chicken = \
    data_country.loc[(data_country['import_chicken_quantity_tons_2017'].isna()) | (data_country['import_chicken_quantity_tons_2013'].isna())]

{{ len(country_no_report_import_chicken)}} countries do have missing information about chicken importations due to the absence of reports in 2013 or 2017:

{{country_no_report_import_chicken.country.tolist()}}

We will get an approximation of these imports through the declaration of exports from partner countries.

In [ ]:
# getting information about the chicken imports on the countries not reporting the imports through the export reports from other countries.

chicken_export = chicken_trade.loc[((chicken_trade['element'] == 'Export Quantity') | (chicken_trade['element'] == 'Export Value')) \
                                  & (chicken_trade['unit'] != '1000 Head')]

# Export of chicken to countries not reporting any chicken imports
chicken_export_to_countries_noreport = chicken_export.loc[chicken_export['partner_country'].isin(country_no_report_import_chicken.country)]

chicken_export_to_countries_noreport = chicken_export_to_countries_noreport.pivot_table(\
                                                                                       index=['partner_country_code', 'partner_country'], columns=['year', 'element'], values=['value'], aggfunc=sum)

chicken_export_to_countries_noreport.columns = ['import_chicken_quantity_tons_2013', 'import_chicken_value_thousand_usd_2013', \
                                                'import_chicken_quantity_tons_2017', 'import_chicken_value_thousand_usd_2017']
chicken_export_to_countries_noreport.reset_index(inplace=True)

chicken_export_to_countries_noreport.rename(columns = {'partner_country_code':'country_code', 'partner_country':'country'}, inplace=True)

In [ ]:
data_country.set_index('country', inplace=True)
chicken_export_to_countries_noreport.set_index('country', inplace=True)

#overlapping values from chicken export to data_country for the countries not reporting their imports
data_country = data_country.combine_first(chicken_export_to_countries_noreport)
data_country.reset_index(inplace=True)

In [ ]:
data_country['import_chicken_value_growth_percent'] = \
    (data_country['import_chicken_value_thousand_usd_2017'] / data_country['import_chicken_value_thousand_usd_2013'] - 1) *100

data_country['import_chicken_quantity_growth_percent'] = \
    (data_country['import_chicken_quantity_tons_2017'] / data_country['import_chicken_quantity_tons_2013'] - 1) *100

data_country['import_chicken_ratio_usd_tons'] = \
    data_country['import_chicken_value_thousand_usd_2017'] * 1000 / data_country['import_chicken_quantity_tons_2013']

Analysis is made on 2017 figures and growth 2013-2017, removing columns with 2013 information

In [ ]:
data_country = data_country.drop([ 'pop_2013_thousand', 'food_supply_kcal_2013',
       'food_supply_kcal_2017', 'food_supply_kcalcapitaday_2013',
       'food_supply_kcalcapitaday_2017', 'gdp_percapita_usd_2013',
       'gdp_percapita_usd_2017', 'import_chicken_quantity_tons_2013',
       'import_chicken_value_thousand_usd_2013',
       'prevalence_protein_animal_capita_percent_2013',
       'prevalence_protein_animal_percent_2013',
       'protein_supply_animal_gcapitaday_2013',
       'protein_supply_animal_growth_percent', 'protein_supply_animal_kg_2013',
       'protein_supply_gcapitaday_2013',
       'protein_supply_kg_2013', 'import_poultry_tons_2013', 'food_poultry_tons_2013', 
        'food_poultry_tons_2013', 'protein_supply_poultry_gcapitaday_2013',  'domestic_supply_poultry_tons_2013'], axis = 1)

In [ ]:
data_country.columns

In [ ]:
data_country

In [ ]:
# Poultry- proteins supply, proteins per capita, imports
temp = data.loc[data['item'] == 'Poultry Meat'].pivot_table(index=['country_code', 'country'], columns=['year'], \
                                                            values=['food', 'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['import_poultry_1000tons_2013', 'import_poultry_1000tons_2017', 'protein_supply_poultry_kg_2013',  'protein_supply_poultry_kg_2017',\
                'protein_supply_poultry_gcapitaday_2013', 'protein_supply_poultry_gcapitaday_2017']
temp.reset_index(inplace=True)
data_country = pd.merge(data_country, temp, how='left')

#growth Poultry+Eggs protein supply
data_country['protein_supply_poultry_growth'] = (data_country['protein_supply_poultry_kg_2017'] / data_country['protein_supply_poultry_kg_2013'] - 1) * 100
#growth Poultry+Eggs imports
data_country['import_poultry_growth'] = (data_country['import_poultry_1000tons_2017'] / data_country['import_poultry_1000tons_2013'] - 1) * 100